In [ ]:
#import require libraries:

# *** Should we import these inside functions mixed eview on practices as it is dire to program but if functiond pulled out ad implemented in other code will require this 
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
import re
import os
import json
import codecs
import ast
from io import StringIO

from os import path
#import word and regex functions for checking 
from disallowTermCheck import checkTerms
from disallowRegexCheck import checkRegex
# from unit_testing_nonallowedwords import checkword

In [ ]:
#parameters for disallowed list and url 
notebookList ='This parameter will be passed through papermill'
disallowedTermsList = 'This parameter will be passed through papermill'

In [ ]:
def readNotebook(notebookToReadPath):
    try:
                file = codecs.open(notebookToReadPath, "r")
                source = file.read()
                y = json.loads(source)
                currentNotebook = []
                for x in y['cells']:
                    for x2 in x['source']:
                        currentNotebook.append(x2)
                
                return currentNotebook
    except:
        print("Can't read file")
        return False

In [ ]:
#notebook checker function 
def notebookChecker(disallowedTermsList, notebookList):
    directory = path.dirname(__file__) if "__file__" in locals() else os.getcwd()
    csvStringIO = StringIO(disallowedTermsList)
    sortedList = np.array(pd.read_csv(csvStringIO, sep=",", header=None))
    termIndexKey = {} 
    termWeightKey = {}
    disallowedList = []
    overallTermCount = {}
    for item in sortedList:
        termIndexKey.update({item[1]:item[0]})
        termWeightKey.update({item[1]:item[2]})
        disallowedList.append(item[1])
        
        
    # print(f"index {termIndexKey} \n weight {termWeightKey}")
    # dictionary of word:rule_num and dictionary of word:weight
    regexFile = "regexFile/regexPatterns.csv"
    regexList = pd.read_csv(path.join(directory, regexFile))
    sortedregexList = pd.DataFrame(regexList, columns=['index','pattern','weight'])
    sortedregexList = np.array(sortedregexList)
    patternIndexKey = {}
    patternWeightKey ={}
    
    disallowedregexList = []
    #here we take the regex patterns and ad them to the list to be checked by regex function
    for item in sortedregexList:
        patternIndexKey.update({item[1]:item[0]})
        patternWeightKey.update({item[1]:item[2]})
        disallowedregexList.append(item[1])
    # print(disallowedregexList)
    overallPatternCount = {pattern : 0 for pattern in disallowedregexList}

    checkedNotebooks = {}
    sortedNotebookList = StringIO(notebookList)
    print(sortedNotebookList)
    uncheckedNotebooks = np.array(pd.read_csv(sortedNotebookList, sep=",", header=None))
    print(uncheckedNotebooks)
    # print (f"\n Here we open and read the notebooks due to be open {uncheckedNotebooks} \n")
    

    #loop through files from list

    for uncheckedNotebook in uncheckedNotebooks:
        
        overallScore = 0
        notebook = {}
        # here we check line by line the the terms found which are added to a list 
        notebookToRead = str(f"{path.join(directory, uncheckedNotebook[0])}")
        contents = readNotebook(notebookToRead)
        
        lineNum = 1
        for item in sortedList:
            overallTermCount[item[1]] = 0
        
        overallTermCountByLine = {}
        overallPatternCountByLine = {}
        
        for contentLine in contents:
            # print(f"\n content line {contentLine} \n")
            termResults = checkTerms(contentLine, disallowedList)
            if termResults != False:
                # print(termResults)
                overallTermCountByLine[lineNum] = termResults
                for term in overallTermCount:
                    if termResults[term] > 0 :
                        overallTermCount[term] += termResults[term]
                        overallScore += termResults[term]
            else:
                overallTermCountByLine[lineNum] = 'none found'
            
            regExOutput = checkRegex(disallowedregexList, contentLine)
            overallPatternCountByLine[lineNum] = regExOutput
            for patternsFound in regExOutput:
                overallPatternCount[patternsFound] += regExOutput[patternsFound]
                overallScore += regExOutput[patternsFound]
            lineNum += 1
        print(uncheckedNotebook)
        print(overallTermCount)
        # print(overallPatternCount)
        # print(overallTermCountByLine)
        # # print(overallPatternCountByLine)
        print(overallScore)
        checkedNotebooks.update({str(uncheckedNotebook): {'overallScore': overallScore,'overallTermCount' : str(overallTermCount),'overallTermCountByLine' : str(overallTermCountByLine), 'overallPatternCount' : str(overallPatternCount),'overallPatternCountByLine' : str(overallPatternCountByLine)}})
  
        
    return checkedNotebooks 
        

notebookResults = notebookChecker(disallowedTermsList, notebookList)
# print(notebookResults)

# Here we check results

In [ ]:
totals = {}
# print(notebookResults.keys())
for result in notebookResults:  
    totals[result] = notebookResults[result].get('overallScore')
    # return back to dictionary for graphs
# print(totals)


# short description

In [ ]:
orderedResults = { nb:score for nb,score in sorted(totals.items(), key=lambda item:item[1])}
df = pd.DataFrame.from_dict(orderedResults, orient='index', columns=[('Total Score')])
print(df)
#PLot examples
f = plt.figure()
f.set_figwidth(3)
f.set_figheight(5)
plt.bar(range(len(orderedResults)),list(orderedResults.values()),align='center',color=['r','g','b'])
plt.xticks(range(len(orderedResults)),list(orderedResults.keys()))
plt.xlabel('Notebook Names')
plt.ylabel('Score')
plt.show

print('Regex patterns Found by notebook')
for result in notebookResults:
    terms = ast.literal_eval(notebookResults[result]['overallPatternCount'])
    terms = pd.DataFrame.from_dict(terms, orient='index', columns=[(result)])
    print(terms)

print('Terms Found by notebook')
for result in notebookResults:
    terms = ast.literal_eval(notebookResults[result]['overallTermCount'])
    g = plt.figure()
    g.set_figwidth(3)
    g.set_figheight(5)
    plt.bar(range(len(terms)),list(terms.values()),align='center',color=['r','g','b'])
    plt.xticks(range(len(terms)),list(terms.keys()))
    plt.xlabel(f" {result}")
    plt.ylabel('Score')
    plt.show



# Long description

In [ ]:
print('Regex patterns Found by notebook')
for result in notebookResults:
    print(f"\n{result}")
    patterns = ast.literal_eval(notebookResults[result]['overallPatternCountByLine'])
    # terms = pd.DataFrame.from_dict(terms, orient='index', columns=[(result)])
    for pattern in patterns:
        print(f"{pattern} : {patterns[pattern]} ")